In [36]:
from functools import reduce


class LiteralPacket:
    def __init__(self, input):
        self.input = input
        self.version = int(self.input[0:3], 2)
        self.type = int(self.input[3:6], 2)
        literal = ""
        pos = 6
        while self.input[pos] == '1':
            literal += self.input[pos+1:pos+5]
            pos += 5
        literal += self.input[pos+1:pos+5]
        self.value = int(literal, 2)
        self.length = pos+5

    def total_version(self):
        return self.version

    def __str__(self):
        return f"Literal packet with value {self.value} and version {self.version}"


class OperatorPacket:
    def __init__(self, input):
        self.subpackets = []
        self.input = input
        self.version = int(self.input[0:3], 2)
        self.type = int(self.input[3:6], 2)
        if self.input[6] == '0':
            subpackets_length = int(self.input[7:22], 2)
            self.length = 22 + subpackets_length
            position = 22
            while(position < self.length):
                subpacket = parse_binary(input[position:])
                self.subpackets.append(subpacket)
                position += subpacket.length
        else:
            subpackets_length = int(self.input[7:18], 2)
            position = 18
            self.length = 18
            for _ in range(subpackets_length):
                left_over_input = input[position:]
                if left_over_input.count('1') > 0: 
                    subpacket = parse_binary(left_over_input)
                    self.subpackets.append(subpacket)
                    position += subpacket.length
                    self.length += subpacket.length
                else:
                    self.length += len(left_over_input)

    def __str__(self):
        head = f"Operator packet version {self.version}"
        return "\n".join([head, *[str(sub) for sub in self.subpackets]])

    def total_version(self):
        return reduce(lambda a, sb: a + sb.total_version(), self.subpackets, 0) + self.version


def parse_hex(input):
    binary_number = bin(int(input, 16))[2:]
    binary_number = "0" * ((4 - (len(binary_number) % 4)) % 4) + binary_number
    return parse_binary(binary_number)


def parse_binary(input):
    if input.count("1") == 0:
        return None
    type = int(input[3:6], 2)
    if type == 4:
        return LiteralPacket(input)
    else:
        return OperatorPacket(input)

877

In [24]:
p = parse_hex("38006F45291200")
assert len(p.subpackets) == 2
assert list(map(lambda sp: sp.value, p.subpackets)) == [10,20]
print(p)

Operator packet version 1
Literal packet with value 10 and version 6
Literal packet with value 20 and version 2


In [25]:
p = parse_hex("EE00D40C823060")
assert len(p.subpackets) == 3
assert list(map(lambda sp: sp.value, p.subpackets)) == [1,2,3]
assert p.version == 7
print(p)

Operator packet version 7
Literal packet with value 1 and version 2
Literal packet with value 2 and version 4
Literal packet with value 3 and version 1


In [26]:
p = parse_hex("D2FE28")
assert p.value == 2021
assert p.length == 21
print(p)

Literal packet with value 2021 and version 6


In [27]:
p = parse_hex("8A004A801A8002F478")
print(p)
print(p.total_version())
assert p.total_version() == 16

Operator packet version 4
Operator packet version 1
Operator packet version 5
Literal packet with value 15 and version 6
16


In [28]:
p = parse_hex("A0016C880162017C3686B18A3D4780")
print(p)
assert p.total_version() == 31

Operator packet version 5
Operator packet version 1
Operator packet version 3
Literal packet with value 6 and version 7
Literal packet with value 6 and version 6
Literal packet with value 12 and version 5
Literal packet with value 15 and version 2
Literal packet with value 15 and version 2


In [29]:
p = parse_hex("C0015000016115A2E0802F182340")
print(p)
assert p.total_version() == 23

Operator packet version 6
Operator packet version 0
Literal packet with value 10 and version 0
Literal packet with value 11 and version 6
Operator packet version 4
Literal packet with value 12 and version 7
Literal packet with value 13 and version 0


In [30]:
input = "60552F100693298A9EF0039D24B129BA56D67282E600A4B5857002439CE580E5E5AEF67803600D2E294B2FCE8AC489BAEF37FEACB31A678548034EA0086253B183F4F6BDDE864B13CBCFBC4C10066508E3F4B4B9965300470026E92DC2960691F7F3AB32CBE834C01A9B7A933E9D241003A520DF316647002E57C1331DFCE16A249802DA009CAD2117993CD2A253B33C8BA00277180390F60E45D30062354598AA4008641A8710FCC01492FB75004850EE5210ACEF68DE2A327B12500327D848028ED0046661A209986896041802DA0098002131621842300043E3C4168B12BCB6835C00B6033F480C493003C40080029F1400B70039808AC30024C009500208064C601674804E870025003AA400BED8024900066272D7A7F56A8FB0044B272B7C0E6F2392E3460094FAA5002512957B98717004A4779DAECC7E9188AB008B93B7B86CB5E47B2B48D7CAD3328FB76B40465243C8018F49CA561C979C182723D769642200412756271FC80460A00CC0401D8211A2270803D10A1645B947B3004A4BA55801494BC330A5BB6E28CCE60BE6012CB2A4A854A13CD34880572523898C7EDE1A9FA7EED53F1F38CD418080461B00440010A845152360803F0FA38C7798413005E4FB102D004E6492649CC017F004A448A44826AB9BFAB5E0AA8053306B0CE4D324BB2149ADDA2904028600021909E0AC7F0004221FC36826200FC3C8EB10940109DED1960CCE9A1008C731CB4FD0B8BD004872BC8C3A432BC8C3A4240231CF1C78028200F41485F100001098EB1F234900505224328612AF33A97367EA00CC4585F315073004E4C2B003530004363847889E200C45985F140C010A005565FD3F06C249F9E3BC8280804B234CA3C962E1F1C64ADED77D10C3002669A0C0109FB47D9EC58BC01391873141197DCBCEA401E2CE80D0052331E95F373798F4AF9B998802D3B64C9AB6617080"

In [37]:
p = parse_hex(input)
p.total_version()

877